In [1]:
!python -V

Python 3.13.5


In [2]:
import pandas as pd

%pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org mlflow==3.1.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle

In [4]:
%pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow
from pathlib import Path

try:
    # 尝试获取当前脚本所在目录（适用于直接运行脚本）
    project_root = Path(__file__).parent.resolve()
except NameError:
    # 如果 __file__ 未定义（如在 Notebook 中），使用当前工作目录
    project_root = Path.cwd().resolve()



# 步骤2：构造数据库路径（项目根目录下的mlflow.db）
db_path = project_root / "mlflow.db"

# 步骤3：转换为SQLite的URI格式（自动处理Windows/Linux路径差异）

db_uri = f"sqlite:///{db_path.as_posix()}"
print(f"Using MLflow tracking URI: {db_uri}")

mlflow.set_tracking_uri(db_uri)


# 强制初始化数据库（创建实验）
try:
    mlflow.set_experiment("nyc-taxi-experiment")
    print("实验创建成功，数据库已初始化")
except Exception as e:
    print(f"初始化失败: {e}")
    # 手动检查文件是否创建
    if db_path.exists():
        print(f"文件已创建，但权限不足: {db_path}")
    else:
        print(f"文件未创建，路径可能无效或权限不足")

Using MLflow tracking URI: sqlite:////Users/bytedance/Documents/ByteRAG/experiment_tracking/mlflow.db


2025/07/01 14:23:03 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/01 14:23:03 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


实验创建成功，数据库已初始化


In [7]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df=read_dataframe('./data/taxi_tripdata.csv')
#get df_train and df_val from df
#by randomly splitting the data into 80% train and 20% validation
df_train = df.sample(frac=0.8, random_state=42)
df_val = df.drop(df_train.index)


/var/folders/ff/s79mvhjj31s_y00l2pyf_s580000gn/T/ipykernel_8355/3970424726.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [9]:
len(df_train), len(df_val)

(63278, 15819)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

71.4023870743136

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/taxi_tripdata.csv")
    mlflow.log_param("valid-data-path", "./data/taxi_tripdata.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/Users/bytedance/.local/share/virtualenvs/experiment_tracking-8QLpu8TW/lib/python3.13/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.93637                           
[1]	validation-rmse:8.38430                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/bytedance/.local/share/virtualenvs/experiment_tracking-8QLpu8TW/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:23:04] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:7.58081                           
[3]	validation-rmse:7.18776                           
[4]	validation-rmse:6.99963                           
[5]	validation-rmse:6.90851                           
[6]	validation-rmse:6.85690                           
[7]	validation-rmse:6.82932                           
[8]	validation-rmse:6.81449                           
[9]	validation-rmse:6.80399                           
[10]	validation-rmse:6.79359                          
[11]	validation-rmse:6.77817                          
[12]	validation-rmse:6.76671                          
[13]	validation-rmse:6.76040                          
[14]	validation-rmse:6.75089                          
[15]	validation-rmse:6.74901                          
[16]	validation-rmse:6.74301                          
[17]	validation-rmse:6.73797                          
[18]	validation-rmse:6.73656                          
[19]	validation-rmse:6.73355                          
[20]	valid

/Users/bytedance/.local/share/virtualenvs/experiment_tracking-8QLpu8TW/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:23:18] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[17]	validation-rmse:6.69531                                                   
[18]	validation-rmse:6.68932                                                   
[19]	validation-rmse:6.68702                                                   
[20]	validation-rmse:6.68705                                                   
[21]	validation-rmse:6.68387                                                   
[22]	validation-rmse:6.67821                                                   
[23]	validation-rmse:6.67637                                                   
[24]	validation-rmse:6.67386                                                   
[25]	validation-rmse:6.67249                                                   
[26]	validation-rmse:6.66533                                                   
[27]	validation-rmse:6.66232                                                   
[28]	validation-rmse:6.65948                                                   
[29]	validation-rmse:6.65732            

...

/Users/bytedance/.local/share/virtualenvs/experiment_tracking-8QLpu8TW/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:27:48] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:11.82084                                                    
[2]	validation-rmse:11.41677                                                    
[3]	validation-rmse:11.03766                                                    
[4]	validation-rmse:10.68624                                                    
[5]	validation-rmse:10.36028                                                    
[6]	validation-rmse:10.05710                                                    
[7]	validation-rmse:9.77619                                                     
[8]	validation-rmse:9.51543                                                     
[9]	validation-rmse:9.27500                                                     
[10]	validation-rmse:9.05365                                                    
[11]	validation-rmse:8.84726                                                    
[12]	validation-rmse:8.65950                                                    
[13]	validation-rmse:8.48429

/Users/bytedance/.local/share/virtualenvs/experiment_tracking-8QLpu8TW/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:28:19] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[1]	validation-rmse:11.54952                                                    
[2]	validation-rmse:11.04125                                                    
[3]	validation-rmse:10.57873                                                    
[4]	validation-rmse:10.15948                                                    
[5]	validation-rmse:9.78043                                                     
[6]	validation-rmse:9.43725                                                     
[7]	validation-rmse:9.12826                                                     
[8]	validation-rmse:8.84975                                                     
[9]	validation-rmse:8.60033                                                     
[10]	validation-rmse:8.37588                                                    
[11]	validation-rmse:8.17584                                                    
[12]	validation-rmse:7.99724                                                    
[13]	validation-rmse:7.83762

KeyboardInterrupt: 

In [21]:
mlflow.xgboost.autolog(disable=True)

In [23]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=330,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:11.85278


/Users/bytedance/.local/share/virtualenvs/experiment_tracking-8QLpu8TW/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:38:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[1]	validation-rmse:11.09323
[2]	validation-rmse:10.43034
[3]	validation-rmse:9.85894
[4]	validation-rmse:9.36512
[5]	validation-rmse:8.93988
[6]	validation-rmse:8.57922
[7]	validation-rmse:8.26839
[8]	validation-rmse:8.00591
[9]	validation-rmse:7.78299
[10]	validation-rmse:7.59600
[11]	validation-rmse:7.43683
[12]	validation-rmse:7.30452
[13]	validation-rmse:7.19303
[14]	validation-rmse:7.10180
[15]	validation-rmse:7.02518
[16]	validation-rmse:6.95948
[17]	validation-rmse:6.90571
[18]	validation-rmse:6.86069
[19]	validation-rmse:6.82202
[20]	validation-rmse:6.78987
[21]	validation-rmse:6.76190
[22]	validation-rmse:6.73973
[23]	validation-rmse:6.71976
[24]	validation-rmse:6.70385
[25]	validation-rmse:6.69016
[26]	validation-rmse:6.67770
[27]	validation-rmse:6.66733
[28]	validation-rmse:6.65707
[29]	validation-rmse:6.64983
[30]	validation-rmse:6.64264
[31]	validation-rmse:6.63693
[32]	validation-rmse:6.63106
[33]	validation-rmse:6.62646
[34]	validation-rmse:6.62134
[35]	validation-rmse:

2025/07/01 14:38:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/bytedance/.local/share/virtualenvs/experiment_tracking-8QLpu8TW/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [14:38:21] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/07/01 14:38:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [24]:
run_id='f5b4bd2300ec4943955d253603ec3e29'
model_uri = f"runs:/{run_id}/models_mlflow"
#register model
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-xgboost")

2025/07/01 14:39:20 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/01 14:39:20 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi-xgboost' already exists. Creating a new version of this model...
2025/07/01 14:39:20 WARNING mlflow.tracking._model_registry.fluent: Run with id f5b4bd2300ec4943955d253603ec3e29 has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-a36aed9b6cff45a4863f07e38ef31f0d instead
Created version '2' of model 'nyc-taxi-xgboost'.


<ModelVersion: aliases=[], creation_timestamp=1751351960220, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1751351960220, metrics=None, model_id=None, name='nyc-taxi-xgboost', params=None, run_id='f5b4bd2300ec4943955d253603ec3e29', run_link=None, source='models:/m-a36aed9b6cff45a4863f07e38ef31f0d', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/taxi_tripdata.csv")
        mlflow.log_param("valid-data-path", "./data/taxi_tripdata.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        